In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from pipython import GCSDevice, pitools
import time
import numpy as np
import os
import cv2
from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE



In [ ]:
#This cell connects with the stage controller and initialises all axes to work
#you can query the stage's position with qPOS and move it with MOV

pidevice = GCSDevice('E-725')
pidevice.InterfaceSetupDlg()
print(pidevice.qIDN())
#pidevice.CloseConnection()

pitools.startup(pidevice, stages=['P-561.3DD','P-561.3DD','P-561.3DD'],refmodes='FRF',servostates=True) 

pidevice.qPOS()

In [ ]:
#These are the functions that sum up the rows to make a summed image from the autofocus line and calculate the centre of mass

def COM(p,delta):
    
    return center_of_mass(p-delta)

def summed1(f):

    im = f
    g = np.zeros_like(f[0],'uint32')
    for i in range(len(im)):
        g += im[i]
   
    return g






In [ ]:
#This file points the current directory to the DLL file folder that is required for the thorlabs cameras

import os
import sys


def configure_path():
    is_64bits = sys.maxsize > 2**32
    relative_path_to_dlls = '.' + os.sep + 'dlls' + os.sep

    if is_64bits:
        relative_path_to_dlls += '64_lib'
    else:
        relative_path_to_dlls += '32_lib'

    absolute_path_to_file_directory = os.path.dirname(os.path.abspath(__file__))

    absolute_path_to_dlls = os.path.abspath(absolute_path_to_file_directory + os.sep + relative_path_to_dlls)

    os.environ['PATH'] = absolute_path_to_dlls + os.pathsep + os.environ['PATH']

    try:
        # Python 3.8 introduces a new method to specify dll directory
        os.add_dll_directory(absolute_path_to_dlls)
    except AttributeError:
        pass



try:
    # if on Windows, use the provided setup script to add the DLLs folder to the PATH
    from windows_setup import configure_path
    configure_path()
except ImportError:
    configure_path = None

In [ ]:
#This function measured the position of the sample by calculating an average of several COM measurements

def measure_pos(print1=True):
    frame = camera.get_pending_frame_or_null()
    if frame is not None:
        COM_temp = []
        for k in range(10):
            frame = camera.get_pending_frame_or_null()
            #print("frame #{} received!".format(frame.frame_count))
            frame.image_buffer
            image_buffer_copy = np.copy(frame.image_buffer)
            numpy_shaped_image = image_buffer_copy.reshape(camera.image_height_pixels, camera.image_width_pixels)
            #out_temp.append(numpy_shaped_image)
            sum_temp = summed1(numpy_shaped_image)
            #stemp.append(sum_temp)
            COM_temp.append(COM(sum_temp,min(sum_temp)))
            if print1==True:
                print(np.mean(np.asarray(COM_temp)))
    return np.mean(np.asarray(COM_temp))

#This is a work in progress algorithm to try move the stage to get the sample back into the correct field of view when things move
#not working yet

def focus_correct(sp,delt,sl):

    set_point = sp

    delta = delt

    slope = sl

    zc = pidevice.qPOS()['3']
    lc = measure_pos(print1=False)
    
    current_pos = (zc,lc)

    test = abs(lc-set_point) < delta

    if test == True:
        return
    else:
        zc = (current_pos[1]-set_point)/slope + current_pos[0]

        print(current_pos,slope)
        print((current_pos[1]-set_point)/slope)
        print(zc)

        #zc = current_pos[0]

        pidevice.MOV('3',zc)

        lc = measure_pos(print1=False)

        new_pos = (zc,lc)

        test = abs(lc-set_point) < delta

        if test == True:
            print("Focus Adjusted!")
            return
        else:
            while test == False:
                prev_pos = current_pos
                current_pos = new_pos
                print(prev_pos)
                print(current_pos)
            
                z_change = current_pos[0]-prev_pos[0]
                l_change = current_pos[1]-prev_pos[1]
                desired_change = set_point - prev_pos[1]
            
                diff = (l_change - desired_change)/desired_change
                
                print("Diff ",diff)
                zc = current_pos[0] - z_change*diff

                print(zc)

                pidevice.MOV('3',zc)
                lc = measure_pos(print1=False)

                new_pos = (zc,lc)

                test = abs(lc-set_point) < delta
            print("Focus adjusted!!")
            return
                
                
#This is a calbiration measurement where the stage moves up in steps and the reflected laser is measured        

def cal_slope():
    curr = pidevice.qPOS('3')['3']

    steps = np.linspace(curr-1,curr+1,201)

    zc = pidevice.MOV('3',steps[0])
    time.sleep(0.1)

    COMs = []

    for i in range(len(steps)):
        pidevice.MOV('3',steps[i])
        time.sleep(0.02)
        COMs.append(measure_pos())

    COMs=np.asarray(COMs)

    coef = np.polyfit(steps,COMs,1)
    poly1d_fn = np.poly1d(coef) 

    plt.plot(steps,COMs)
    plt.plot(steps,poly1d_fn(steps))
    plt.show()

    return coef[0]

    
        
        

    


    
    

    
    

In [ ]:
#this cell calibrates the slope, opens the connection to the camera and runs the calibration slope function

pidevice.MOV('3',2)

with TLCameraSDK() as sdk:
    available_cameras = sdk.discover_available_cameras()
    if len(available_cameras) < 1:
        print("no cameras detected")

    with sdk.open_camera(available_cameras[0]) as camera:
        camera.exposure_time_us = 85  # set exposure to 11 ms
        camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
        camera.image_poll_timeout_ms = 1000  # 1 second polling timeout
    
        camera.arm(2)
    
        camera.issue_software_trigger()

        print(measure_pos(print1=False))

        cal_out=cal_slope()

        

    


In [ ]:
#This is the framework for trying to autocorrect the position, not working yet

with TLCameraSDK() as sdk:
    available_cameras = sdk.discover_available_cameras()
    if len(available_cameras) < 1:
        print("no cameras detected")

    with sdk.open_camera(available_cameras[0]) as camera:
        camera.exposure_time_us = 85  # set exposure to 11 ms
        camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
        camera.image_poll_timeout_ms = 1000  # 1 second polling timeout
    
        camera.arm(2)
    
        camera.issue_software_trigger()

        set_point=measure_pos(print1=True)
        print(pidevice.qPOS('3')['3'])
        delta=0.3
        for i in range(50):
            #pi
            curr = measure_pos(print1=False)
            diff1 = abs(curr-set_point)
            test = abs(measure_pos(print1=False)-set_point) < delta
            if test == True:
                print('Fine')
                print(curr,set_point,diff1)
                time.sleep(1)
                continue
            else:
                print(set_point,measure_pos(print1=False))
                print('AA',curr,set_point,diff1)
                focus_correct(set_point,delta,cal_out)
                time.sleep(1)
                

        
            